# Import

In [ ]:
import os
import random

import pandas as pd
import numpy as np

from PIL import Image
from tqdm import tqdm 

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn, optim

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold


if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Using device:", device)

Using device: mps


# Hyperparameter Setting

In [2]:
CFG = {
    'IMG_SIZE': 384,
    'BATCH_SIZE': 16,
    'EPOCHS': 15,
    'LEARNING_RATE': 1e-4,
    'SEED' : 42,
    'N_SPLITS': 3
}

# Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(CFG['SEED'])

# CustomDataset

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None, is_test=False):
        self.root_dir = root_dir
        self.transform = transform
        self.is_test = is_test
        self.samples = []

        if is_test:
            for fname in sorted(os.listdir(root_dir)):
                if fname.lower().endswith(('.jpg')):
                    img_path = os.path.join(root_dir, fname)
                    self.samples.append((img_path,))
        else:
            self.classes = sorted(os.listdir(root_dir))
            self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

            for cls_name in self.classes:
                cls_folder = os.path.join(root_dir, cls_name)

                if not os.path.isdir(cls_folder):
                    continue
                
                for fname in os.listdir(cls_folder):
                    if fname.lower().endswith(('.jpg')):
                        img_path = os.path.join(cls_folder, fname)
                        label = self.class_to_idx[cls_name]
                        self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if self.is_test:
            img_path = self.samples[idx][0]
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image
        else:
            img_path, label = self.samples[idx]
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, label

# Data Load

In [5]:
train_root = './filtered_train'
test_root = './test'

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((CFG['IMG_SIZE'] + 32, CFG['IMG_SIZE'] + 32)),
    transforms.RandomResizedCrop(CFG['IMG_SIZE'], scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [ ]:
full_dataset = CustomImageDataset(train_root, transform=None)
print(f"총 이미지 수: {len(full_dataset)}")

targets = [label for _, label in full_dataset.samples]
class_names = full_dataset.classes

train_idx, val_idx = train_test_split(
    range(len(targets)), test_size=0.2, stratify=targets, random_state=42
)

train_dataset = Subset(CustomImageDataset(train_root, transform=train_transform), train_idx)
val_dataset = Subset(CustomImageDataset(train_root, transform=val_transform), val_idx)
print(f'train 이미지 수: {len(train_dataset)}, valid 이미지 수: {len(val_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)


총 이미지 수: 33131
train 이미지 수: 26504, valid 이미지 수: 6627


# ConvNeXt

In [ ]:
import torch.nn as nn
import timm

class BaseModel(nn.Module):
    def __init__(self, num_classes):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model('convnext_base_384_in22ft1k', pretrained=True, features_only=False)
        self.feature_dim = self.backbone.head.in_features
        self.backbone.head = nn.Identity()
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.head = nn.Linear(self.feature_dim, num_classes)

    def forward(self, x):
        x = self.backbone.forward_features(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.head(x)
        return x

/Users/hyun/venv/torch_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def mixup_data(x, y, alpha=0.2):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [ ]:
def cutmix_data(x, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    batch_size, _, H, W = x.size()
    index = torch.randperm(batch_size).to(x.device)

    cx = np.random.randint(W)
    cy = np.random.randint(H)
    cut_w = int(W * np.sqrt(1 - lam))
    cut_h = int(H * np.sqrt(1 - lam))

    x1 = np.clip(cx - cut_w // 2, 0, W)
    x2 = np.clip(cx + cut_w // 2, 0, W)
    y1 = np.clip(cy - cut_h // 2, 0, H)
    y2 = np.clip(cy + cut_h // 2, 0, H)

    x[:, :, y1:y2, x1:x2] = x[index, :, y1:y2, x1:x2]
    y_a, y_b = y, y[index]
    lam = 1 - ((x2 - x1) * (y2 - y1) / (W * H))
    return x, y_a, y_b, lam


def apply_mixup_or_cutmix(x, y, mix_prob=0.5, mixup_alpha=0.2, cutmix_alpha=1.0):
    if np.random.rand() < mix_prob:
        return mixup_data(x, y, alpha=mixup_alpha)
    else:
        return cutmix_data(x, y, alpha=cutmix_alpha)

# Train/ Validation

In [ ]:
def load_checkpoint(model, optimizer, scheduler, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch']
    best_logloss = checkpoint.get('best_logloss', float('inf'))
    best_acc = checkpoint.get('best_acc', 0.0)
    best_ce_loss = checkpoint.get('best_ce_loss', float('inf'))
    return start_epoch, best_logloss, best_acc, best_ce_loss

resume = True
resume_ckpt = {
    0: "final_model_fold1/checkpoint_epoch_005.pth",
    1: None,
    2: None,
    3: None,
    4: None
}

skf = StratifiedKFold(n_splits=CFG['N_SPLITS'], shuffle=True, random_state=42)
targets = [label for _, label in full_dataset.samples]
class_names = full_dataset.classes

for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(targets)), targets)):
    print(f"\n📂 Fold {fold+1}/{CFG['N_SPLITS']}")

    train_dataset = Subset(CustomImageDataset(train_root, transform=train_transform), train_idx)
    val_dataset = Subset(CustomImageDataset(train_root, transform=val_transform), val_idx)
    train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)


    model = BaseModel(num_classes=len(class_names)).to(device)
    optimizer = optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'])
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG['EPOCHS'], eta_min=1e-6)
    criterion = nn.CrossEntropyLoss()

    save_dir = f"final_model_fold{fold+1}"
    os.makedirs(save_dir, exist_ok=True)

    start_epoch = 0
    best_logloss = float('inf')
    best_acc = 0.0
    best_ce_loss = float('inf')
    if resume and resume_ckpt.get(fold):
        print(f"🔄 Resuming fold {fold+1} from {resume_ckpt[fold]}")
        start_epoch, best_logloss, best_acc, best_ce_loss = load_checkpoint(
            model, optimizer, scheduler, resume_ckpt[fold]
        )
        print(f"Resume: start_epoch={start_epoch}, best_logloss={best_logloss}, best_acc={best_acc}, best_ce_loss={best_ce_loss}")

    for epoch in range(start_epoch, CFG['EPOCHS']):
        model.train()
        train_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"[Fold {fold+1}][Epoch {epoch+1}/{CFG['EPOCHS']}] Training"):
            images, labels = images.to(device), labels.to(device)
            inputs, targets_a, targets_b, lam = apply_mixup_or_cutmix(
                images, labels, mix_prob=0.5, mixup_alpha=0.2, cutmix_alpha=1.0
            )

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        all_probs = []
        all_labels = []

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"[Fold {fold+1}][Epoch {epoch+1}/{CFG['EPOCHS']}] Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

                probs = F.softmax(outputs, dim=1)
                all_probs.extend(probs.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * correct / total
        val_logloss = log_loss(all_labels, all_probs, labels=list(range(len(class_names))))
        scheduler.step()

        print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.2f}% | LogLoss: {val_logloss:.4f}")

        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_logloss': best_logloss,
            'best_acc': best_acc,
            'best_ce_loss': best_ce_loss,
        }
        torch.save(checkpoint, f"{save_dir}/checkpoint_epoch_{epoch+1:03d}.pth")

        if val_logloss < best_logloss:
            best_logloss = val_logloss
            torch.save(model.state_dict(), f"{save_dir}/best_logloss.pth")

        if val_accuracy > best_acc:
            best_acc = val_accuracy
            torch.save(model.state_dict(), f"{save_dir}/best_acc.pth")

        if avg_val_loss < best_ce_loss:
            best_ce_loss = avg_val_loss
            torch.save(model.state_dict(), f"{save_dir}/best_loss.pth")



📂 Fold 1/3


/Users/hyun/venv/torch_venv/lib/python3.12/site-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name convnext_base_384_in22ft1k to current convnext_base.fb_in22k_ft_in1k_384.
  model = create_fn(


🔄 Resuming fold 1 from final_model_fold1/checkpoint_epoch_005.pth
Resume: start_epoch=5, best_logloss=0.15354506109472799, best_acc=95.19195943498733, best_ce_loss=0.15338741486953428


[Fold 1][Epoch 6/15] Validation: 100%|██████████| 691/691 [05:29<00:00,  2.10it/s]
/Users/hyun/venv/torch_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Train Loss: 0.9392 | Val Loss: 0.1390 | Val Acc: 95.93% | LogLoss: 0.1392


...

[Fold 2][Epoch 15/15] Validation: 100%|██████████| 691/691 [05:57<00:00,  1.93it/s]
/Users/hyun/venv/torch_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Train Loss: 0.6549 | Val Loss: 0.0805 | Val Acc: 97.65% | LogLoss: 0.0806

📂 Fold 3/3


/Users/hyun/venv/torch_venv/lib/python3.12/site-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name convnext_base_384_in22ft1k to current convnext_base.fb_in22k_ft_in1k_384.
  model = create_fn(
[Fold 3][Epoch 1/15] Validation: 100%|██████████| 691/691 [05:18<00:00,  2.17it/s]
/Users/hyun/venv/torch_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Train Loss: 4.0209 | Val Loss: 0.6242 | Val Acc: 83.41% | LogLoss: 0.6248


[Fold 3][Epoch 2/15] Training:   5%|▌         | 70/1381 [36:45<9:18:59, 25.58s/it] 

In [ ]:
def load_checkpoint(model, optimizer, scheduler, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch']
    best_logloss = checkpoint.get('best_logloss', float('inf'))
    best_acc = checkpoint.get('best_acc', 0.0)
    best_ce_loss = checkpoint.get('best_ce_loss', float('inf'))
    return start_epoch, best_logloss, best_acc, best_ce_loss

resume = True
resume_ckpt = {
    0: None,
    1: None,
    2: "final_model_fold3/checkpoint_epoch_001.pth",
    3: None,
    4: None
}

skf = StratifiedKFold(n_splits=CFG['N_SPLITS'], shuffle=True, random_state=42)
targets = [label for _, label in full_dataset.samples]
class_names = full_dataset.classes

for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(targets)), targets)):
    if fold != 2:
        continue
    
    print(f"\n📂 Fold {fold+1}/{CFG['N_SPLITS']}")

    train_dataset = Subset(CustomImageDataset(train_root, transform=train_transform), train_idx)
    val_dataset = Subset(CustomImageDataset(train_root, transform=val_transform), val_idx)
    train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

    model = BaseModel(num_classes=len(class_names)).to(device)
    optimizer = optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'])
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG['EPOCHS'], eta_min=1e-6)
    criterion = nn.CrossEntropyLoss()

    save_dir = f"final_model_fold{fold+1}"
    os.makedirs(save_dir, exist_ok=True)

    start_epoch = 0
    best_logloss = float('inf')
    best_acc = 0.0
    best_ce_loss = float('inf')
    if resume and resume_ckpt.get(fold):
        print(f"🔄 Resuming fold {fold+1} from {resume_ckpt[fold]}")
        start_epoch, best_logloss, best_acc, best_ce_loss = load_checkpoint(
            model, optimizer, scheduler, resume_ckpt[fold]
        )
        print(f"Resume: start_epoch={start_epoch}, best_logloss={best_logloss}, best_acc={best_acc}, best_ce_loss={best_ce_loss}")

    for epoch in range(start_epoch, CFG['EPOCHS']):
        model.train()
        train_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"[Fold {fold+1}][Epoch {epoch+1}/{CFG['EPOCHS']}] Training"):
            images, labels = images.to(device), labels.to(device)
            inputs, targets_a, targets_b, lam = apply_mixup_or_cutmix(
                images, labels, mix_prob=0.5, mixup_alpha=0.2, cutmix_alpha=1.0
            )

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        all_probs = []
        all_labels = []

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"[Fold {fold+1}][Epoch {epoch+1}/{CFG['EPOCHS']}] Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

                probs = F.softmax(outputs, dim=1)
                all_probs.extend(probs.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * correct / total
        val_logloss = log_loss(all_labels, all_probs, labels=list(range(len(class_names))))
        scheduler.step()

        print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.2f}% | LogLoss: {val_logloss:.4f}")

        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_logloss': best_logloss,
            'best_acc': best_acc,
            'best_ce_loss': best_ce_loss,
        }
        torch.save(checkpoint, f"{save_dir}/checkpoint_epoch_{epoch+1:03d}.pth")

        if val_logloss < best_logloss:
            best_logloss = val_logloss
            torch.save(model.state_dict(), f"{save_dir}/best_logloss.pth")

        if val_accuracy > best_acc:
            best_acc = val_accuracy
            torch.save(model.state_dict(), f"{save_dir}/best_acc.pth")

        if avg_val_loss < best_ce_loss:
            best_ce_loss = avg_val_loss
            torch.save(model.state_dict(), f"{save_dir}/best_loss.pth")


📂 Fold 3/3


/Users/hyun/venv/torch_venv/lib/python3.12/site-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name convnext_base_384_in22ft1k to current convnext_base.fb_in22k_ft_in1k_384.
  model = create_fn(


🔄 Resuming fold 3 from final_model_fold3/checkpoint_epoch_001.pth
Resume: start_epoch=1, best_logloss=inf, best_acc=0.0, best_ce_loss=inf


[Fold 3][Epoch 2/15] Validation: 100%|██████████| 691/691 [05:13<00:00,  2.21it/s]
/Users/hyun/venv/torch_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Train Loss: 1.5755 | Val Loss: 0.2542 | Val Acc: 92.61% | LogLoss: 0.2544


...

Train Loss: 0.6732 | Val Loss: 0.0866 | Val Acc: 97.23% | LogLoss: 0.0867


[Fold 3][Epoch 14/15] Validation: 100%|██████████| 691/691 [05:08<00:00,  2.24it/s]
/Users/hyun/venv/torch_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Train Loss: 0.6783 | Val Loss: 0.0844 | Val Acc: 97.41% | LogLoss: 0.0845


[Fold 3][Epoch 15/15] Validation: 100%|██████████| 691/691 [05:08<00:00,  2.24it/s]
/Users/hyun/venv/torch_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Train Loss: 0.6560 | Val Loss: 0.0838 | Val Acc: 97.39% | LogLoss: 0.0839


# Inference

In [15]:
test_dataset = CustomImageDataset(test_root, transform=None, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

In [ ]:
IMG_SIZE = CFG['IMG_SIZE']
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
BATCH_SIZE = CFG['BATCH_SIZE']


def get_tta_transforms():
    return [
        transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.RandomHorizontalFlip(p=1.0),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.RandomRotation(degrees=5),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
    ]

tta_transforms = get_tta_transforms()

fold_list = [1, 2, 3]
model_paths = [
    f'/Users/hyun/dev_ws/dacon_car/final_model_fold{fold}/checkpoint_epoch_015.pth'
    for fold in fold_list
]

ensemble_probs = []

for fold_idx, path in enumerate(model_paths):
    model = BaseModel(num_classes=len(class_names))
    checkpoint = torch.load(path, map_location=device)

    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint)
    model.to(device)
    model.eval()

    with torch.no_grad():
        for tta_idx, tta_tf in enumerate(tta_transforms):
            probs_list = []

            for images in tqdm(test_loader, desc=f"Fold {fold_idx+1}, TTA {tta_idx+1}"):
                transformed = [tta_tf(img) for img in images]
                transformed = torch.stack(transformed).to(device)

                outputs = model(transformed)
                probs = F.softmax(outputs, dim=1)
                probs_list.append(probs.cpu())

            all_probs = torch.cat(probs_list, dim=0)
            ensemble_probs.append(all_probs)

avg_probs = torch.stack(ensemble_probs).mean(dim=0)

results = [
    {class_names[i]: prob[i].item() for i in range(len(class_names))}
    for prob in avg_probs
]
pred = pd.DataFrame(results)

pred['label'] = pred[class_names].idxmax(axis=1)
pred['ID'] = [f'TEST_{i:05d}' for i in range(len(pred))]

submission = pred[['ID', 'label']]
submission.to_csv("ensemble_3fold_3tta_submission.csv", index=False)
print("✅ 3-Fold × 3-TTA soft voting 완료 및 저장 완료")

Fold 3, TTA 3: 100%|██████████| 517/517 [04:07<00:00,  2.09it/s]


✅ 3-Fold × 3-TTA soft voting 완료 및 저장 완료


# Submission

In [ ]:
submission = pd.read_csv('./sample_submission.csv', encoding='utf-8-sig')

class_columns = submission.columns[1:]
pred = pred[class_columns]

submission[class_columns] = pred.values
submission.to_csv('test36_submission.csv', index=False, encoding='utf-8-sig')